In [1]:
import pandas as pd
import datetime
import os
import json
from fuzzywuzzy import fuzz
import numpy as np

/usr/local/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
json_file="/home/jian/Docs/Geo_mapping/dma_map_latest.json"
DMA_name_mapping=json.load(open(json_file,"r"))

In [3]:
def dma_guess(dma, reference = DMA_name_mapping, max_only = True, threshold = 0):
    dma = dma.strip().upper().capitalize()
    _reference = set(reference.values())
    _results = {}
    for i in _reference:
        if ((i[:len(dma)] == dma) & (len(dma) >= 10)):
            _results.update({101:i})
            _results.update({fuzz.ratio(dma, i): i})
        else:
            _results.update({fuzz.ratio(dma, i): i})
    if len(_results) > 0:
        _results_filtered = {k:v for (k,v) in _results.items() if k > threshold}
        if max_only:
            return _results_filtered[max(_results.keys())]
        elif max_only == False:
            return _results_filtered
 
       
def dma_match(target, reference = DMA_name_mapping, output = True, wild_card = {'National': 'National'}):
    _pdma = [] # paired
    msgs = []
    for dma in target:
        try:
            #msgs.append("Found EXACT Match from previous records '{}': '{}'".format(dma,reference[dma]))
            _pdma.append(reference[dma.strip()])
        except KeyError:
            _pdma.append(dma_guess(dma, reference, max_only = True))
            msgs.append("'{}': '{}'".format(dma,dma_guess(dma, reference, max_only = True)))
            #print "Found match for '{}': '{}".format(dma, dma_guess(dma, DMA_name_mapping, True))
    if not msgs:
        print('All records found in DMA_name_mapping, please proceed. ')
    else:
        print ("Suggested NEW Match: {")
        for i in list(set(msgs)):
            print (i+',')
        print ("}")
 
        print ("Run the following code if new matches are corrent: \n DMA_name_mapping.update({")
        for i in list(set(msgs)):
            print (i+',')
        print ("})")

In [4]:
all_values=[]
for key_dma in list(DMA_name_mapping.keys()):
    all_values=list(set(all_values+[DMA_name_mapping[key_dma]]))

In [5]:
allocation_pctg=pd.read_csv("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/from_DMA_to_IRI_based_on_common_population.csv",dtype=str)
allocation_pctg=allocation_pctg[['DMA','IRI Market','Pctg']]
allocation_pctg['Pctg']=allocation_pctg['Pctg'].apply(lambda x: float(x))

In [6]:
agg_df=pd.DataFrame()
for dma in allocation_pctg['DMA'].unique().tolist():
    if dma not in DMA_name_mapping.keys():
        print(dma)
        for std_dma in all_values:
            if fuzz.ratio(dma, std_dma)>10:
                df=pd.DataFrame({"found_dma":dma,"std_dma":std_dma,"ratio":fuzz.ratio(dma, std_dma)},index=[dma])
                agg_df=agg_df.append(df).sort_values("ratio",ascending=False).head(5)

In [7]:
with open(json_file, 'w') as outfile:  
    json.dump(DMA_name_mapping, outfile)

In [8]:
data_folder=os.getcwd()+"/data/"

In [9]:
data_folder

'/home/jian/SunnyD/Allocate_DMA_spend_to_IRI/data/'

In [10]:
TMR_cols=['DMA','Date','cost_DMA','Media','coefficient','raw_spend_type']

# TV

In [11]:
# 1. Local
spend_1_TV_local=pd.read_excel(data_folder+"Update_Sep_1st_Week/"+"SunnyD August Local Cable & Broadcast Spend by day by DMA - Final - 9.14.18.xlsx",dtype=str,sheetname="Cable",skiprows=1)

local_tv_cols=spend_1_TV_local.columns.tolist()
local_tv_cols=['DMA']+[str(x)[:10] for x in local_tv_cols[1:len(local_tv_cols)-1]]+['DMA Total']

spend_1_TV_local.columns=local_tv_cols

spend_1_TV_local=spend_1_TV_local[local_tv_cols[:len(local_tv_cols)-1]]
for col in spend_1_TV_local.columns.tolist()[1:]:
    spend_1_TV_local[col]=spend_1_TV_local[col].astype(float)
spend_1_TV_local=spend_1_TV_local[spend_1_TV_local['DMA']!='nan']
   
spend_1_TV_local=spend_1_TV_local.melt(id_vars="DMA",var_name="Date",value_name="cost_DMA")
spend_1_TV_local['Media']='TV_Local'
spend_1_TV_local['coefficient']=0.9958
spend_1_TV_local['raw_spend_type']='gross'
spend_1_TV_local=spend_1_TV_local[['Date','DMA','Media','coefficient','raw_spend_type','cost_DMA']]
spend_1_TV_local['cost_DMA'].sum()


229974.5

In [12]:
# 2. Broadcast
spend_2_TV_broadcast=pd.read_excel(data_folder+"Update_Sep_1st_Week/"+"SunnyD August Local Cable & Broadcast Spend by day by DMA - Final - 9.14.18.xlsx",dtype=str,sheetname="Broadcast",skiprows=1)

broadcast_tv_cols=spend_2_TV_broadcast.columns.tolist()
broadcast_tv_cols=['DMA']+[str(x)[:10] for x in broadcast_tv_cols[1:len(broadcast_tv_cols)-1]]+['DMA Total']

spend_2_TV_broadcast.columns=broadcast_tv_cols

spend_2_TV_broadcast=spend_2_TV_broadcast[broadcast_tv_cols[:len(local_tv_cols)-1]]
for col in spend_2_TV_broadcast.columns.tolist()[1:]:
    spend_2_TV_broadcast[col]=spend_2_TV_broadcast[col].astype(float)
spend_2_TV_broadcast=spend_2_TV_broadcast[spend_2_TV_broadcast['DMA']!='nan']
   
spend_2_TV_broadcast=spend_2_TV_broadcast.melt(id_vars="DMA",var_name="Date",value_name="cost_DMA")
spend_2_TV_broadcast['Media']='TV_Broadcast'
spend_2_TV_broadcast['coefficient']=0.9958
spend_2_TV_broadcast['raw_spend_type']='gross'
spend_2_TV_broadcast=spend_2_TV_broadcast[['Date','DMA','Media','coefficient','raw_spend_type','cost_DMA']]
spend_2_TV_broadcast['cost_DMA'].sum()
spend_2_TV_broadcast['DMA']=spend_2_TV_broadcast['DMA'].apply(lambda x: x.replace(" Total",""))


In [13]:
local_TV_cost=spend_1_TV_local.append(spend_2_TV_broadcast)
local_TV_cost.head(2)

,Date,DMA,Media,coefficient,raw_spend_type,cost_DMA
0,2018-07-31,Atlanta,TV_Local,0.9958,gross,2498.0
1,2018-07-31,Augusta,TV_Local,0.9958,gross,215.0


In [14]:
agg_df=pd.DataFrame()
for dma in local_TV_cost['DMA'].unique().tolist():
    if dma not in DMA_name_mapping.keys():
        print(dma)
        for std_dma in all_values:
            if fuzz.ratio(dma, std_dma)>10:
                df=pd.DataFrame({"found_dma":dma,"std_dma":std_dma,"ratio":fuzz.ratio(dma, std_dma)},index=[dma])
                agg_df=agg_df.append(df).sort_values("ratio",ascending=False).head(5)

In [15]:
# If empty above, no need to run
'''
updated_dict=agg_df.head(4)[['std_dma']].reset_index().rename(columns={"index":"DMA_Found"})
updated_dict.index=updated_dict['DMA_Found']
del updated_dict['DMA_Found']
updated_dict=updated_dict.to_dict()
DMA_name_mapping.update(updated_dict['std_dma'])
'''

'\nupdated_dict=agg_df.head(4)[[\'std_dma\']].reset_index().rename(columns={"index":"DMA_Found"})\nupdated_dict.index=updated_dict[\'DMA_Found\']\ndel updated_dict[\'DMA_Found\']\nupdated_dict=updated_dict.to_dict()\nDMA_name_mapping.update(updated_dict[\'std_dma\'])\n'

In [16]:
local_TV_cost.head(2)

,Date,DMA,Media,coefficient,raw_spend_type,cost_DMA
0,2018-07-31,Atlanta,TV_Local,0.9958,gross,2498.0
1,2018-07-31,Augusta,TV_Local,0.9958,gross,215.0


In [17]:
local_TV_cost.head(2)

,Date,DMA,Media,coefficient,raw_spend_type,cost_DMA
0,2018-07-31,Atlanta,TV_Local,0.9958,gross,2498.0
1,2018-07-31,Augusta,TV_Local,0.9958,gross,215.0


# National TV

In [18]:
spend_national_TV=pd.read_excel("/home/jian/SunnyD/Allocate_DMA_spend_to_IRI/data/Update_Sep_1st_Week/Galavision and UniMas Spend by Day by DMA_1.xlsx",dtype=str,skiprows=9)
spend_national_TV_dates=[str(x)[:10] for x in spend_national_TV.columns.tolist() if "Unnamed" not in str(x)]
spend_national_TV=pd.read_excel("/home/jian/SunnyD/Allocate_DMA_spend_to_IRI/data/Galavision and UniMas Spend by Day by DMA.xlsx",dtype=str,skiprows=10)
spend_national_TV=spend_national_TV.rename(columns={"Unnamed: 0":"DMA"})
spend_national_TV=spend_national_TV[[x for x in spend_national_TV.columns.tolist() if "Spend" not in x]]
spend_national_TV=spend_national_TV.iloc[:,0:len(spend_national_TV_dates)*2+1]

In [19]:
columns_name=[]
for date_str in spend_national_TV_dates:
    for channel in ["Galavision","UniMas"]:
        name_str=date_str+"|"+channel
        columns_name=columns_name+[name_str]
spend_national_TV.columns=['DMA']+columns_name

In [20]:
spend_national_TV_G=spend_national_TV[['DMA']+[col for col in spend_national_TV.columns.tolist() if "Gala" in col]]
spend_national_TV_U=spend_national_TV[['DMA']+[col for col in spend_national_TV.columns.tolist() if "Uni" in col]]
spend_national_TV_G.columns=['DMA']+[x[:10] for x in spend_national_TV_G.columns.tolist()[1:]]
spend_national_TV_U.columns=['DMA']+[x[:10] for x in spend_national_TV_U.columns.tolist()[1:]]
spend_national_TV_G=spend_national_TV_G.melt(id_vars="DMA",var_name="Date",value_name="cost_DMA")
spend_national_TV_U=spend_national_TV_U.melt(id_vars="DMA",var_name="Date",value_name="cost_DMA")
spend_national_TV_G['Media']="National_TV_Galavision"
spend_national_TV_U['Media']="National_TV_UniMas"
national_TV_cost=spend_national_TV_G.append(spend_national_TV_U)
national_TV_cost['coefficient']=0.9958
national_TV_cost['raw_spend_type']="gross"
national_TV_cost=national_TV_cost[local_TV_cost.columns.tolist()]
national_TV_cost['DMA']=national_TV_cost['DMA'].apply(lambda x: x.split(" (")[0])
national_TV_cost['cost_DMA']=national_TV_cost['cost_DMA'].astype(float)

# FB/IG

In [21]:
FB_Inst_cost=pd.read_excel(data_folder+"Update_Sep_1st_Week/"+"FB-IG_Sunny D_Spend by DMA Day_9.4.2018.xlsx",dtype=str,sheetname='Raw Data Report',skiprows=2)
FB_Inst_cost=FB_Inst_cost[~FB_Inst_cost['Day'].isin(['nan','All'])]
FB_Inst_cost=FB_Inst_cost[['DMA Region','Day','Amount Spent']].rename(columns={"DMA Region":"DMA","Day":"Date","Amount Spent":"cost_DMA"})
FB_Inst_cost['cost_DMA']=FB_Inst_cost['cost_DMA'].astype(float)
FB_Inst_cost['Media']='FB_IG'
FB_Inst_cost['coefficient']=0.9958/0.85
FB_Inst_cost['raw_spend_type']='net'
FB_Inst_cost=FB_Inst_cost[TMR_cols]

In [22]:
agg_df=pd.DataFrame()
for dma in FB_Inst_cost['DMA'].unique().tolist():
    if dma not in DMA_name_mapping.keys():
        print(dma)
        for std_dma in all_values:
            if fuzz.ratio(dma, std_dma)>10:
                df=pd.DataFrame({"found_dma":dma,"std_dma":std_dma,"ratio":fuzz.ratio(dma, std_dma)},index=[dma])
                agg_df=agg_df.append(df).sort_values("ratio",ascending=False).head(5)

# Snapchat

In [23]:
Snapchat_cost=pd.read_excel(data_folder+"Update_Sep_1st_Week/"+"Snapchat_ad-account-EST-3157_SunnyD_SC-campaigns-data.xlsx",dtype=str)
Snapchat_cost=Snapchat_cost[['Start time','Spend']].rename(columns={"Start time":"Date","Spend":"cost_DMA"})
Snapchat_cost['DMA']='xx'
Snapchat_cost['Media']="Snapchat"
Snapchat_cost['Date']=Snapchat_cost['Date'].apply(lambda x: x[:10])
Snapchat_cost['coefficient']=0.9958/0.85
Snapchat_cost['cost_DMA']=Snapchat_cost['cost_DMA'].astype(float)
Snapchat_cost['Date']=Snapchat_cost['Date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Snapchat_cost['raw_spend_type']='net'
Snapchat_cost=Snapchat_cost[local_TV_cost.columns]
Snapchat_cost['Date']=Snapchat_cost['Date'].astype(str)

# Spotify

In [24]:
# Already included in Digital

'''
Spotify_cost_date=pd.read_excel(data_folder+"3158_SunnyD_2018Campaign_Spotify PG.xlsx",dtype=str,sheetname="Spotify")
rows=Spotify_cost_date[Spotify_cost_date['DMA'].apply(lambda x: "00:" in x)].reset_index()['index'].tolist()+[len(Spotify_cost_date)]

Spotify_cost=pd.DataFrame()
for i in range(len(rows)-1):
    df=Spotify_cost_date.iloc[rows[i]:rows[i+1],]
    date=Spotify_cost_date['DMA'][rows[i]][:10]
    df['Date']=date
    df=df.reset_index()
    del df['index']
    df=df.iloc[1:]
    Spotify_cost=Spotify_cost.append(df)
Spotify_cost['DMA']=Spotify_cost['DMA'].replace("nan","xx")

Spotify_cost['Date']=Spotify_cost['Date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Spotify_cost['cost_DMA']=Spotify_cost['Media Dollars'].apply(lambda x: float(x))
Spotify_cost['Media']='Spotify'
Spotify_cost['coefficient']=0.9958/0.85
Spotify_cost['raw_spend_type']="net"
Spotify_cost=Spotify_cost[local_TV_cost.columns.tolist()]
'''

'\nSpotify_cost_date=pd.read_excel(data_folder+"3158_SunnyD_2018Campaign_Spotify PG.xlsx",dtype=str,sheetname="Spotify")\nrows=Spotify_cost_date[Spotify_cost_date[\'DMA\'].apply(lambda x: "00:" in x)].reset_index()[\'index\'].tolist()+[len(Spotify_cost_date)]\n\nSpotify_cost=pd.DataFrame()\nfor i in range(len(rows)-1):\n    df=Spotify_cost_date.iloc[rows[i]:rows[i+1],]\n    date=Spotify_cost_date[\'DMA\'][rows[i]][:10]\n    df[\'Date\']=date\n    df=df.reset_index()\n    del df[\'index\']\n    df=df.iloc[1:]\n    Spotify_cost=Spotify_cost.append(df)\nSpotify_cost[\'DMA\']=Spotify_cost[\'DMA\'].replace("nan","xx")\n\nSpotify_cost[\'Date\']=Spotify_cost[\'Date\'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())\nSpotify_cost[\'cost_DMA\']=Spotify_cost[\'Media Dollars\'].apply(lambda x: float(x))\nSpotify_cost[\'Media\']=\'Spotify\'\nSpotify_cost[\'coefficient\']=0.9958/0.85\nSpotify_cost[\'raw_spend_type\']="net"\nSpotify_cost=Spotify_cost[local_TV_cost.columns.tolist()]\

# Cinema

In [25]:
Cinema_cost=pd.read_excel(data_folder+"HH SunnyD Cinema Breakout by Day_8.24_FOR JUBA.XLSX",dtype=str,skiprows=3,sheetname="Spend by DMA_Cinema")
Cinema_cost=Cinema_cost.iloc[:,1:(Cinema_cost.shape[1]-1)]

In [26]:
Cinema_cost_header=[x[:10] for x in [str(col) for col in Cinema_cost.columns.tolist()] if "-" in x]
Cinema_cost.columns=[['DMA']+Cinema_cost_header]
# All DMA found

Cinema_cost=pd.melt(Cinema_cost, id_vars='DMA', value_vars=Cinema_cost.columns.tolist()[1:])
Cinema_cost.columns=['DMA','Date','cost_DMA']
Cinema_cost['cost_DMA']=Cinema_cost['cost_DMA'].astype(float)
Cinema_cost=Cinema_cost.groupby(['DMA','Date'])['cost_DMA'].sum().to_frame().reset_index()
Cinema_cost['Media']='Cinema'
Cinema_cost['coefficient']=0.9958/0.85
Cinema_cost['raw_spend_type']='net'
Cinema_cost=Cinema_cost[local_TV_cost.columns]

# Dish_DTV

In [27]:
DISH_cost=pd.read_excel(data_folder+"Update_Sep_1st_Week/"+"HH SunnyD DISH & DTV Breakout by Day_9.4_FOR JUBA.xlsx",dtype=str,skiprows=3,sheetname="Spend by DMA_DISH")
DISH_cost=DISH_cost.iloc[:,1:(DISH_cost.shape[1]-1)]
DISH_cost.head(2)


DISH_cost_header=[x[:10] for x in [str(col) for col in DISH_cost.columns.tolist()] if "-" in x]
DISH_cost.columns=[['DMA']+DISH_cost_header]
DISH_cost=pd.melt(DISH_cost, id_vars='DMA', value_vars=DISH_cost.columns.tolist()[1:])
DISH_cost.columns=['DMA','Date','cost_DMA']
DISH_cost['cost_DMA']=DISH_cost['cost_DMA'].astype(float)
DISH_cost=DISH_cost.groupby(['DMA','Date'])['cost_DMA'].sum().to_frame().reset_index()
DISH_cost['Media']='DISH'
DISH_cost['coefficient']=0.9958
DISH_cost['raw_spend_type']='gross'
DISH_cost=DISH_cost[local_TV_cost.columns]

In [28]:
DTV_cost=pd.read_excel(data_folder+"Update_Sep_1st_Week/"+"HH SunnyD DISH & DTV Breakout by Day_9.4_FOR JUBA.xlsx",dtype=str,skiprows=3,sheetname="Spend by DMA_DTV")
DTV_cost=DTV_cost.iloc[:,1:(DTV_cost.shape[1]-1)]
DTV_cost.head(2)

DTV_cost_header=[x[:10] for x in [str(col) for col in DTV_cost.columns.tolist()] if "-" in x]
DTV_cost.columns=[['DMA']+DTV_cost_header]
DTV_cost=pd.melt(DTV_cost, id_vars='DMA', value_vars=DTV_cost.columns.tolist()[1:])
DTV_cost.columns=['DMA','Date','cost_DMA']
DTV_cost['cost_DMA']=DTV_cost['cost_DMA'].astype(float)
DTV_cost=DTV_cost.groupby(['DMA','Date'])['cost_DMA'].sum().to_frame().reset_index()
DTV_cost['Media']='DTV'
DTV_cost['coefficient']=0.9958
DTV_cost['raw_spend_type']='gross'
DTV_cost=DTV_cost[local_TV_cost.columns]

# Digital by zip

In [57]:
Digital_cost=pd.read_excel(data_folder+"Update_Sep_1st_Week/"+"Estimated Digital Spend By Zip By Day_Sunny D 9.11.xlsx",
                           dtype=str,sheetname="Total Spend by Zip by Day")
Digital_cost_header=[x for x in [str(col) for col in Digital_cost.columns.tolist()] if "/" in x]
Digital_cost.columns=[['Zips']+Digital_cost_header+["Total"]]
Digital_cost=Digital_cost.fillna(0)
Digital_cost=Digital_cost.rename(columns={"Zips":"zip_cd"})

In [58]:
Digital_cost=Digital_cost[Digital_cost['zip_cd']!='Grand Total']
Digital_cost=Digital_cost[Digital_cost['zip_cd']!='Total']
Digital_cost['zip_cd']=Digital_cost['zip_cd'].apply(lambda x: x.zfill(5))

In [59]:
zip_IRI_mapping=pd.read_excel("/home/jian/SunnyD/Otherinput/All_IRI_Mkt_with_Zips_0430.xlsx",dtype=str)

In [60]:
Digital_cost=pd.merge(Digital_cost,zip_IRI_mapping,on="zip_cd",how="left")
Digital_cost['IRI Market']=Digital_cost['IRI Market'].fillna("NO MARKET")
# del Digital_cost['zip_cd']

QC_Digital_cost=Digital_cost[['zip_cd','IRI Market']+[x for x in Digital_cost.columns.tolist() if "/" in x]]
QC_Digital_cost['18_Mkt_Ind']=Digital_cost['IRI Market'].apply(lambda x: In_18(x))
QC_Digital_cost.to_csv("/home/jian/SunnyD/Allocate_DMA_spend_to_IRI/QC_Digital.csv",index=False)

Digital_cost=Digital_cost[['IRI Market']+[x for x in Digital_cost.columns.tolist() if "/" in x]]

In [33]:
Digital_cost=pd.melt(Digital_cost, id_vars='IRI Market', value_vars=Digital_cost.columns.tolist()[1:])
Digital_cost.columns=['IRI Market','Date','cost_IRI']
Digital_cost['cost_IRI']=Digital_cost['cost_IRI'].astype(float)
Digital_cost=Digital_cost.groupby(['IRI Market','Date'])['cost_IRI'].sum().to_frame().reset_index()
Digital_cost['Media']='Digital'
Digital_cost['coefficient']=1
Digital_cost['adjusted_cost_IRI']=Digital_cost['cost_IRI']*Digital_cost['coefficient']
Digital_cost['raw_spend_type']='adjusted'
Digital_cost['Date']=Digital_cost['Date'].apply(lambda x: datetime.date(int(x.split("/")[2]),int(x.split("/")[0]),int(x.split("/")[1])))
Digital_cost['Date']=Digital_cost['Date'].astype(str)


# Pandora by Zip

In [35]:
# Already included in Digital
'''
Pandora_cost=pd.read_excel(data_folder+"Pandora Estimated Digital Spend By Zip By Day_Sunny D.xlsx",dtype=str,sheetname="PANDORA TOTAL ESTIMATED SPEND")
Pandora_cost_header=[x[:10] for x in [str(col) for col in Pandora_cost.columns.tolist()] if "-" in x]
Pandora_cost.columns=[['zip_cd']+Pandora_cost_header+["Total"]]
Pandora_cost=Pandora_cost[Pandora_cost['zip_cd']!='Total']
Pandora_cost['zip_cd']=Pandora_cost['zip_cd'].apply(lambda x: x.zfill(5))

Pandora_cost=pd.merge(Pandora_cost,zip_IRI_mapping,on="zip_cd",how="left")
Pandora_cost['IRI Market']=Pandora_cost['IRI Market'].fillna("NO MARKET")
del Pandora_cost['zip_cd']

Pandora_cost=Pandora_cost[['IRI Market']+[x for x in Pandora_cost.columns.tolist() if "-" in x]]

Pandora_cost=pd.melt(Pandora_cost, id_vars='IRI Market', value_vars=Pandora_cost.columns.tolist()[1:])
Pandora_cost.columns=['IRI Market','Date','cost_IRI']
Pandora_cost['cost_IRI']=Pandora_cost['cost_IRI'].astype(float)
Pandora_cost=Pandora_cost.groupby(['IRI Market','Date'])['cost_IRI'].sum().to_frame().reset_index()
Pandora_cost['Media']='Pandora'
Pandora_cost['coefficient']=0.9958
Pandora_cost['adjusted_cost_IRI']=Pandora_cost['cost_IRI']*Pandora_cost['coefficient']
Pandora_cost['raw_spend_type']='gross'
'''

'\nPandora_cost=pd.read_excel(data_folder+"Pandora Estimated Digital Spend By Zip By Day_Sunny D.xlsx",dtype=str,sheetname="PANDORA TOTAL ESTIMATED SPEND")\nPandora_cost_header=[x[:10] for x in [str(col) for col in Pandora_cost.columns.tolist()] if "-" in x]\nPandora_cost.columns=[[\'zip_cd\']+Pandora_cost_header+["Total"]]\nPandora_cost=Pandora_cost[Pandora_cost[\'zip_cd\']!=\'Total\']\nPandora_cost[\'zip_cd\']=Pandora_cost[\'zip_cd\'].apply(lambda x: x.zfill(5))\n\nPandora_cost=pd.merge(Pandora_cost,zip_IRI_mapping,on="zip_cd",how="left")\nPandora_cost[\'IRI Market\']=Pandora_cost[\'IRI Market\'].fillna("NO MARKET")\ndel Pandora_cost[\'zip_cd\']\n\nPandora_cost=Pandora_cost[[\'IRI Market\']+[x for x in Pandora_cost.columns.tolist() if "-" in x]]\n\nPandora_cost=pd.melt(Pandora_cost, id_vars=\'IRI Market\', value_vars=Pandora_cost.columns.tolist()[1:])\nPandora_cost.columns=[\'IRI Market\',\'Date\',\'cost_IRI\']\nPandora_cost[\'cost_IRI\']=Pandora_cost[\'cost_IRI\'].astype(float)\nPan

# Append all of the 3 DMA files

In [36]:
# Add digital later
TMR_cost=Snapchat_cost.append(FB_Inst_cost).append(local_TV_cost).append(national_TV_cost).append(Cinema_cost).append(DISH_cost).append(DTV_cost)

In [37]:
TMR_cost[TMR_cost['cost_DMA'].apply(lambda x: type(x))==str]['Media'].unique()

array([], dtype=object)

In [38]:
TMR_cost['std_DMA']=TMR_cost['DMA'].apply(lambda x: DMA_name_mapping[x])
del TMR_cost['DMA']
TMR_cost['adjusted_cost_DMA']=TMR_cost['cost_DMA']*TMR_cost['coefficient']

# Apply Pctg

In [39]:
mapping_pctg=pd.read_csv("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/from_DMA_to_IRI_based_on_common_population.csv",dtype=str)
mapping_pctg=mapping_pctg.append(pd.DataFrame({"IRI Market":"National","Pctg":1,"DMA":"National"},index=[0]))
mapping_pctg=mapping_pctg.append(pd.DataFrame({"IRI Market":"Unknown","Pctg":1,"DMA":"Unknown"},index=[0]))
mapping_pctg=mapping_pctg.append(pd.DataFrame({"IRI Market":["NO MARKET"]*4,"Pctg":[1]*4,"DMA":['Honolulu, HI','Anchorage, AK',
                                                                                                'Fairbanks, AK','Juneau, AK']},index=[0]*4))

mapping_pctg=mapping_pctg[['DMA','IRI Market','Pctg']]
mapping_pctg['std_DMA']=mapping_pctg['DMA'].apply(lambda x: DMA_name_mapping[x])
del mapping_pctg['DMA']
mapping_pctg['Pctg']=mapping_pctg['Pctg'].astype(float)

In [40]:
TMR_cost_IRI=pd.merge(TMR_cost,mapping_pctg,on='std_DMA',how="left")

In [41]:
TMR_cost_IRI['adjusted_cost_IRI']=TMR_cost_IRI['adjusted_cost_DMA']*TMR_cost_IRI['Pctg']

In [42]:
# Add digital

In [43]:
TMR_cost_IRI=TMR_cost_IRI.append(Digital_cost)

In [53]:
Markets_18=['BIRMINGHAM/MONTGOMERY','NEW ORLEANS/MOBILE','PHOENIX/TUCSON','LOS ANGELES','DENVER','MIAMI/FT LAUDERDALE',
            'ATLANTA','SOUTH CAROLINA','CHICAGO','BOSTON','BALTIMORE/WASHINGTON','DETROIT','MISSISSIPPI','RALEIGH/GREENSBORO',
            'NEW YORK','WEST TEX/NEW MEXICO','HOUSTON','DALLAS/FT WORTH']

In [51]:
def In_18(x):
    if x in Markets_18:
        y="In_18_Mkts"
    else:
        y="Other"
    return y

In [46]:
TMR_cost_IRI['18_Mkt_Ind']=TMR_cost_IRI['IRI Market'].apply(lambda x: In_18(x))

In [47]:
TMR_cost_IRI['IRI Market']=np.where(TMR_cost_IRI['std_DMA']=="National","National",
                               np.where(TMR_cost_IRI['std_DMA'].isin(['xx','Honolulu, HI']),"Unknown",TMR_cost_IRI['IRI Market']))

In [48]:
TMR_cost_IRI.to_csv(os.getcwd()+"/output/SunnyD_allocated_medial_spend_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [49]:
func= {'adjusted_cost_IRI':['sum'], 'Date':['max','min']}
TMR_cost_IRI['Date']=TMR_cost_IRI['Date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
summary=TMR_cost_IRI.groupby(['IRI Market','18_Mkt_Ind','Media']).agg(func).reset_index()
summary.columns=summary.columns.get_level_values(0)
summary.columns=summary.columns.tolist()[0:4]+['Date_Max','Date_Min']
summary['Cost_Pctg']=summary['adjusted_cost_IRI']/summary['adjusted_cost_IRI'].sum()

In [50]:
summary=summary.sort_values(['IRI Market','Cost_Pctg'],ascending=[True,False])
summary.to_csv(os.getcwd()+"/output/SunnyD_allocated_medial_spend_summary_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [51]:
dates_in_data=TMR_cost_IRI.groupby(['Media','raw_spend_type','coefficient']).agg(func).reset_index()
dates_in_data.columns=dates_in_data.columns.get_level_values(0)
dates_in_data.columns=dates_in_data.columns.tolist()[0:4]+['Date_Max','Date_Min']
dates_in_data['Cost_Pctg']=dates_in_data['adjusted_cost_IRI']/dates_in_data['adjusted_cost_IRI'].sum()

In [52]:
# TMR_cost_IRI['Date']=TMR_cost_IRI['Date'].astype(str)
dates_in_data.to_csv(os.getcwd()+"/output/SunnyD_media_date_range_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [53]:
IRI_summary_writer=pd.ExcelWriter(os.getcwd()+"/output/SunnyD_media_spend_by_IRI_JL"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
summary.to_excel(IRI_summary_writer,"IRI_Level_All_Cost_summary",index=False)
TMR_cost_IRI.to_excel(IRI_summary_writer,"IRI_Level_All_Cost",index=False)
dates_in_data.to_excel(IRI_summary_writer,"dates_in_data_IRI",index=False)
IRI_summary_writer.save()

In [54]:
TMR_cost_IRI['Date'].apply(lambda x: type(x)).unique()

array([<class 'datetime.date'>], dtype=object)

In [55]:
TMR_cost_IRI[TMR_cost_IRI['Date'].apply(lambda x: type(x))==datetime.date]['Media'].unique()

array(['Snapchat', 'FB_IG', 'TV_Local', 'TV_Broadcast',
       'National_TV_Galavision', 'National_TV_UniMas', 'Cinema', 'DISH',
       'DTV', 'Digital'], dtype=object)

# DMA level

In [56]:
DMA_Level_Cost=Snapchat_cost.append(FB_Inst_cost).append(local_TV_cost).append(national_TV_cost).append(Cinema_cost).append(DISH_cost).append(DTV_cost)
DMA_Level_Cost.head(2)


,DMA,Date,Media,coefficient,cost_DMA,raw_spend_type
0,xx,2018-08-14,Snapchat,1.171529,173.02,net
1,xx,2018-08-15,Snapchat,1.171529,2488.68,net


In [57]:
'''
Pandora_cost=pd.read_excel(data_folder+"Pandora Estimated Digital Spend By Zip By Day_Sunny D.xlsx",dtype=str,sheetname="PANDORA TOTAL ESTIMATED SPEND")
Pandora_cost_header=[x[:10] for x in [str(col) for col in Pandora_cost.columns.tolist()] if "-" in x]
Pandora_cost.columns=[['zip_cd']+Pandora_cost_header+["Total"]]
Pandora_cost=Pandora_cost[Pandora_cost['zip_cd']!='Total']
Pandora_cost['zip_cd']=Pandora_cost['zip_cd'].apply(lambda x: x.zfill(5))
Pandora_cost=pd.melt(Pandora_cost, id_vars='zip_cd', value_vars=Pandora_cost.columns.tolist()[1:])
Pandora_cost=Pandora_cost.rename(columns={"variable":"Date","value":"cost"})
Pandora_cost=Pandora_cost[Pandora_cost['Date']!="Total"]

'''

'\nPandora_cost=pd.read_excel(data_folder+"Pandora Estimated Digital Spend By Zip By Day_Sunny D.xlsx",dtype=str,sheetname="PANDORA TOTAL ESTIMATED SPEND")\nPandora_cost_header=[x[:10] for x in [str(col) for col in Pandora_cost.columns.tolist()] if "-" in x]\nPandora_cost.columns=[[\'zip_cd\']+Pandora_cost_header+["Total"]]\nPandora_cost=Pandora_cost[Pandora_cost[\'zip_cd\']!=\'Total\']\nPandora_cost[\'zip_cd\']=Pandora_cost[\'zip_cd\'].apply(lambda x: x.zfill(5))\nPandora_cost=pd.melt(Pandora_cost, id_vars=\'zip_cd\', value_vars=Pandora_cost.columns.tolist()[1:])\nPandora_cost=Pandora_cost.rename(columns={"variable":"Date","value":"cost"})\nPandora_cost=Pandora_cost[Pandora_cost[\'Date\']!="Total"]\n\n'

In [58]:
Digital_cost=pd.read_excel(data_folder+"Update_Sep_1st_Week/"+"Estimated Digital Spend By Zip By Day_Sunny D 9.11.xlsx",
                           dtype=str,sheetname="Total Spend by Zip by Day")
Digital_cost_header=[x for x in [str(col) for col in Digital_cost.columns.tolist()] if "/" in x]
Digital_cost.columns=[['Zips']+Digital_cost_header+["Total"]]
Digital_cost=Digital_cost.fillna(0)
Digital_cost=Digital_cost.rename(columns={"Zips":"zip_cd"})
Digital_cost=Digital_cost[Digital_cost['zip_cd']!='Grand Total']
Digital_cost=Digital_cost[Digital_cost['zip_cd']!='Total']
Digital_cost['zip_cd']=Digital_cost['zip_cd'].apply(lambda x: x.zfill(5))
Digital_cost=pd.melt(Digital_cost, id_vars='zip_cd', value_vars=Digital_cost.columns.tolist()[1:])
Digital_cost=Digital_cost.rename(columns={"variable":"Date","value":"cost"})
Digital_cost=Digital_cost[Digital_cost['zip_cd']!="Total"]
Digital_cost=Digital_cost[Digital_cost['Date']!="Total"]

In [59]:
zip_DMA_allocated=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",
                                dtype=str,skiprows=1,usecols=['CODE','NAME'])
zip_DMA_allocated.columns=[['zip_cd','DMA']]
zip_DMA_allocated['DMA']=zip_DMA_allocated['DMA'].apply(lambda x: DMA_name_mapping[x])

In [60]:
zip_DMA_allocated_share=zip_DMA_allocated.groupby(['zip_cd'])['DMA'].count().to_frame().reset_index()
zip_DMA_allocated_share=zip_DMA_allocated_share.rename(columns={"DMA":"share"})
zip_DMA_allocated_share['share']=zip_DMA_allocated_share['share'].apply(lambda x: 1/x)
zip_DMA_allocated=pd.merge(zip_DMA_allocated,zip_DMA_allocated_share,on="zip_cd",how="left")

In [61]:
# Pandora_cost_DMA=pd.merge(Pandora_cost,zip_DMA_allocated,on='zip_cd',how="outer")
Digital_cost_DMA=pd.merge(Digital_cost,zip_DMA_allocated,on='zip_cd',how="outer")

Digital_cost_DMA['DMA']=Digital_cost_DMA['DMA'].fillna("xx")
Digital_cost_DMA['share']=Digital_cost_DMA['share'].fillna(1)

# Pandora_cost_DMA['cost']=Pandora_cost_DMA['cost'].astype(float)
Digital_cost_DMA['cost']=Digital_cost_DMA['cost'].astype(float)

# Pandora_cost_DMA['cost_DMA']=Pandora_cost_DMA['cost']*Pandora_cost_DMA['share']
Digital_cost_DMA['cost_DMA']=Digital_cost_DMA['cost']*Digital_cost_DMA['share']

# Pandora_cost_DMA=Pandora_cost_DMA[~pd.isnull(Pandora_cost_DMA['cost'])]
Digital_cost_DMA=Digital_cost_DMA[~pd.isnull(Digital_cost_DMA['cost'])]

In [62]:

# Pandora_cost_DMA['Date']=Pandora_cost_DMA['Date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Digital_cost_DMA['Date']=Digital_cost_DMA['Date'].apply(lambda x: datetime.date(int(x.split("/")[2]),int(x.split("/")[0]),int(x.split("/")[1])))
Digital_cost_DMA['Date']=Digital_cost_DMA['Date'].astype(str)
# Pandora_cost_DMA=Pandora_cost_DMA.groupby(['DMA','Date'])['cost_DMA'].sum().to_frame().reset_index()
Digital_cost_DMA=Digital_cost_DMA.groupby(['DMA','Date'])['cost_DMA'].sum().to_frame().reset_index()


In [63]:
'''
Pandora_cost_DMA['Media']="Pandora"
Pandora_cost_DMA['coefficient']=0.9958
Pandora_cost_DMA['raw_spend_type']='gross'
Pandora_cost_DMA=Pandora_cost_DMA[DMA_Level_Cost.columns.tolist()]
'''
Digital_cost_DMA['Media']="Digital"
Digital_cost_DMA['coefficient']=1
Digital_cost_DMA['raw_spend_type']='adjusted'
Digital_cost_DMA=Digital_cost_DMA[DMA_Level_Cost.columns.tolist()]

In [64]:
DMA_Level_All_Cost=DMA_Level_Cost.append(Digital_cost_DMA)
DMA_Level_All_Cost['std_DMA']=DMA_Level_All_Cost['DMA'].apply(lambda x: DMA_name_mapping[x])
DMA_Level_All_Cost['adjusted_DMA_cost']=DMA_Level_All_Cost['cost_DMA']*DMA_Level_All_Cost['coefficient']

In [65]:
DMA_Level_All_Cost_summary=DMA_Level_All_Cost.groupby(['std_DMA'])['adjusted_DMA_cost'].sum().to_frame().reset_index()
DMA_Level_All_Cost_summary=DMA_Level_All_Cost_summary.sort_values(['adjusted_DMA_cost'],ascending=False)

In [66]:
DMA_summary_writer=pd.ExcelWriter(os.getcwd()+"/output/SunnyD_media_spend_by_DMA_JL"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
DMA_Level_All_Cost_summary.to_excel(DMA_summary_writer,"DMA_Level_All_Cost_summary",index=False)
DMA_Level_All_Cost.to_excel(DMA_summary_writer,"DMA_Level_All_Cost",index=False)


In [68]:
func_DMA={'adjusted_DMA_cost': ['sum'],'Date': ['max', 'min']}
DMA_Level_All_Cost['Date']=DMA_Level_All_Cost['Date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())

dates_in_data_DMA=DMA_Level_All_Cost.groupby(['Media','raw_spend_type','coefficient']).agg(func_DMA).reset_index()
dates_in_data_DMA.columns=dates_in_data_DMA.columns.get_level_values(0)
dates_in_data_DMA.columns=dates_in_data_DMA.columns.tolist()[0:4]+['Date_Max','Date_Min']
dates_in_data_DMA['Cost_Pctg']=dates_in_data_DMA['adjusted_DMA_cost']/dates_in_data_DMA['adjusted_DMA_cost'].sum()
dates_in_data_DMA.to_excel(DMA_summary_writer,"dates_in_data_DMA",index=False)
DMA_summary_writer.save()